In [1]:
import json
import os
import time
import pandas as pd
from gensim.models import KeyedVectors 
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Flatten

Using TensorFlow backend.


In [2]:
with open("huffpo_train<20.json") as f: #all huffpo comments of length <20 words
    data=json.load(f)
sample = data[:1000]

In [3]:
sample[0]

['gia',
 'mapp',
 'that',
 'nags',
 'the',
 'hell',
 'out',
 'of',
 'me',
 'christians',
 'what',
 'bible',
 'are',
 'they',
 'reading']

In [4]:
t = Tokenizer(oov_token='<unk>')
t.fit_on_texts(sample)

In [5]:
t.__dir__()


['word_counts',
 'word_docs',
 'filters',
 'split',
 'lower',
 'num_words',
 'document_count',
 'char_level',
 'oov_token',
 'index_docs',
 'word_index',
 'index_word',
 '__module__',
 '__doc__',
 '__init__',
 'fit_on_texts',
 'fit_on_sequences',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'texts_to_matrix',
 'sequences_to_matrix',
 'get_config',
 'to_json',
 '__dict__',
 '__weakref__',
 '_tf_api_names',
 '_tf_api_names_v1',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [6]:
a = t.texts_to_sequences_generator(sample)

In [7]:
#seq_sample = list(map(lambda comment: t.texts_to_sequences(comment), sample))
seq_sample = [t.texts_to_sequences(x) for x in sample]
seq_data = list()
for seq in seq_sample:
    flat = [num for sublist in seq for num in sublist]
    seq_data.append(flat)

In [8]:
seq_data[2]

[8, 420, 587, 58, 128, 258, 320, 22, 45, 421, 48, 1035, 1036, 1037]

In [9]:
t.sequences_to_texts(seq_data[:2])

['gia mapp that nags the hell out of me christians what bible are they reading',
 'this is starting to be an annoying bandwagon now']

In [10]:
max_length = max([len(seq) for seq in seq_data])
sequences = pad_sequences(seq_data, maxlen=max_length, padding='pre')

print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 19


In [11]:
sequences[0]

array([   0,    0,    0,    0, 1029, 1030,   10, 1031,    2,  319,   47,
          9,   68,  586,   27, 1032,   15,   22,  257], dtype=int32)

In [12]:

# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
vocab_sz = len(t.word_index)+1
print('vocab:',vocab_sz)
y_cat = to_categorical(y, num_classes=vocab_sz)


vocab: 3003


In [13]:
print("X[0]->{}\ny[0]->{}\ny_cat[0]->{}".format(X[0],y[0],y_cat[0]))

X[0]->[   0    0    0    0 1029 1030   10 1031    2  319   47    9   68  586
   27 1032   15   22]
y[0]->257
y_cat[0]->[0. 0. 0. ... 0. 0. 0.]


In [14]:
print("X[1].{}.\ny[1].{}.\ny_cat[1].{}".format(X[1],y[1],y_cat[1]))

X[1].[   0    0    0    0    0    0    0    0    0    0   23    6  419    5
   18   34 1033 1034].
y[1].90.
y_cat[1].[0. 0. 0. ... 0. 0. 0.]


In [15]:
y_cat.shape

(1000, 3003)

In [16]:
len(y_cat[0])

3003

In [17]:
print(vocab_sz)

3003


In [18]:
# define model
model = Sequential()
model.add(Embedding(vocab_sz, 100, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_sz, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 100)           300300    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_1 (Dense)              (None, 3003)              153153    
Total params: 483,653
Trainable params: 483,653
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network

if (X.ndim == 1):
    X = numpy.array([X])
    
model.fit(X, y_cat,batch_size=10, epochs=100, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 4s - loss: 7.8946 - acc: 0.0200
Epoch 2/100
 - 3s - loss: 6.8445 - acc: 0.0230
Epoch 3/100
 - 3s - loss: 6.5276 - acc: 0.0230
Epoch 4/100
 - 3s - loss: 6.3786 - acc: 0.0230
Epoch 5/100
 - 3s - loss: 6.2414 - acc: 0.0230
Epoch 6/100
 - 3s - loss: 6.1055 - acc: 0.0230
Epoch 7/100
 - 3s - loss: 5.9724 - acc: 0.0230
Epoch 8/100
 - 3s - loss: 5.8498 - acc: 0.0310
Epoch 9/100
 - 3s - loss: 5.7258 - acc: 0.0360
Epoch 10/100
 - 3s - loss: 5.5993 - acc: 0.0340
Epoch 11/100
 - 3s - loss: 5.4632 - acc: 0.0380
Epoch 12/100
 - 3s - loss: 5.3329 - acc: 0.0460
Epoch 13/100
 - 3s - loss: 5.1933 - acc: 0.0520
Epoch 14/100
 - 3s - loss: 5.0560 - acc: 0.0560
Epoch 15/100
 - 3s - loss: 4.9220 - acc: 0.0640
Epoch 16/100
 - 3s - loss: 4.7952 - acc: 0.0670
Epoch 17/100
 - 3s - loss: 4.6614 - acc: 0.0870
Epoch 18/100
 - 3s - loss: 4.5324 - acc: 0.1020
Epoch 19/100
 - 3s - loss: 4.4066 - acc: 0.1170
Epoch 20/100
 - 3s - loss: 4.2818 - acc: 0.1500
E

In [20]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text


In [21]:
print(generate_seq(model, t, max_length-1, 'him', 20))

him much retarded cheat cheat proud proud money china hahaha lmfao marriage charlottesville a picture honor insane insane snicker bud snicker


In [22]:
print(generate_seq(model, t, max_length-1, 'trump is the worst', 20))

trump is the worst research time ago ago ago ago ago there him him like like right right right it please hahahahah disabled touch
